# Working with dbachecks Historical Results

This guide will assist you to get used to the database schema where dbachecks historical reports are stored if you have followed the blog post [dbachecks – Save the results to a database for historical reporting](https://sqldbawithabeard.com/2018/05/23/dbachecks-save-the-results-to-a-database-for-historical-reporting/)

The first thing that is required is to update the data in the database so that it reflects the current date as if dbachecks had been run today


## Demo Database and container

I have provided you with a demo backup which you can find in my [GitHub Presentations Repository](https://github.com/SQLDBAWithABeard/Presentations/tree/master/Notebooks/dbachecks-historical).

Download this backup file and save it in a folder, I am going to assume that you are using `C:\dbachecks` for the rest of this notebook. I am also going to assume that you have Docker For Windows on your laptop (This will work in Docker running on other OS's but you will have to alter the paths accordingly). You will also find a docker-compose file in the [Repository](https://github.com/SQLDBAWithABeard/Presentations/blob/master/Notebooks/dbachecks-historical/docker-compose.yml) which you can use to spin up a SQL container and restore the backup. (Yes, I know I could have created a container with the datbase already embedded but this gives you more things to learn about!)

The docker-compose file looks like this
````
version: '3.7'

services:
    HistoricalReporting:
        image: mcr.microsoft.com/mssql/server:2019-CTP2.4-ubuntu
        ports:  
          - "15594:1433"
        environment:
          SA_PASSWORD: "Password0!"
          ACCEPT_EULA: "Y"
        volumes: 
          - C:\dbachecks:/var/opt/mssql/backups
````
It will create 

- a single service (a container (See the [Docker Documentation for a full definition of a service](https://docs.docker.com/get-started/part3/))) called `HistoricalReporting`
- assign port 1433 on the container to port 15594 on the host
- add the environmental variables for the SA Password and the acceptance of the EULA
- map the directory c:\dbachecks on the host (your laptop) to the directory /var/opt/mssql/backups in the container

It will create the container using the Community Technical Preview 2.4 image of SQL Server 2019 running on Ubuntu from the Microsoft Container Registry for SQL Server.

You can, and should, understand how to change those values in the docker-compose file for different values so that you can spin up a container with a different image (maybe a later or earlier versions of SQL Server) or store your files in a different directory to `C:\dbachecks` or becuase you are allergic to `Password0!`

Once your docker-compose.yml file is ready open PowerShell (Windows or Core both will work) and type

````
Set-Locatioon c:\dbachecks
docker-compose up -d
````


In [ ]:
version: '3.7'

services:
    2019-CTP24:
        image: mcr.microsoft.com/mssql/server:2019-CTP2.4-ubuntu
        ports:  
          - "15594:1433"
        environment:
          SA_PASSWORD: "Password0!"
          ACCEPT_EULA: "Y"
        volumes: 
          - D:\Git:/var/opt/mssql/backups

In [1]:
/*Resets the date for the demo*/
USE [ValidationResults]
GO

UPDATE [dbachecks].[Prod_dbachecks_summary]
   SET [TestDate] = GETDATE()
 WHERE SummaryID = 7
GO

UPDATE [dbachecks].[Prod_dbachecks_summary]
   SET [TestDate] = DATEADD(day,-1,GetDAte())
 WHERE SummaryID = 6
GO

UPDATE [dbachecks].[Prod_dbachecks_summary]
   SET [TestDate] = DATEADD(day,-2,GetDAte())
 WHERE SummaryID = 5
 GO
 
UPDATE [dbachecks].[Prod_dbachecks_summary]
   SET [TestDate] = DATEADD(day,-3,GetDAte())
 WHERE SummaryID = 4
 GO
 
UPDATE [dbachecks].[Prod_dbachecks_summary]
   SET [TestDate] = DATEADD(day,-4,GetDAte())
 WHERE SummaryID = 3
 GO

 
UPDATE [dbachecks].[Prod_dbachecks_summary]
   SET [TestDate] = DATEADD(day,-5,GetDAte())
 WHERE SummaryID = 2
 GO

 
UPDATE [dbachecks].[Prod_dbachecks_summary]
   SET [TestDate] = DATEADD(day,-6,GetDAte())
 WHERE SummaryID = 1
 GO


Lets list the Checks that we have and the descriptions

In [3]:
USE [ValidationResults]
GO
SELECT 
UniqueTag AS 'Check',  
Description
FROM dbachecks.Checks

I want to see the total number of checks that passed and failed by date

In [4]:
USE [ValidationResults]
GO
SELECT 
TestDate,
TotalCount AS 'Total Checks',
PassedCount AS Passed,
FailedCount AS Failed,
SkippedCount AS Skipped
FROM dbachecks.Prod_dbachecks_summary


What are todays failures ?

In [6]:
SELECT 
d.Name,
ErrorRecord
FROM dbachecks.Prod_dbachecks_detail d
JOIN
dbachecks.Prod_dbachecks_summary s
ON
d.SummaryID = S.SummaryID
WHERE
s.TestDate = Convert(date, GetDate())
AND
d.Passed = 0
ORDER BY d.Name


Which instance had the most failures today ?

In [7]:
; With CTE AS (
SELECT 
DetailID,
RIGHT(Context, charindex(' ', reverse(Context) + ' ') - 1) AS SqlInstance
FROM dbachecks.Prod_dbachecks_detail
) 
SELECT 
CTE.SqlInstance,
COUNT(d.DetailID) AS 'Failed Checks'
FROM dbachecks.Prod_dbachecks_detail d
JOIN CTE
ON CTE.DetailID = d.DetailID
JOIN
dbachecks.Prod_dbachecks_summary s
ON
d.SummaryID = S.SummaryID
WHERE
s.TestDate = Convert(date, GetDate())
AND
d.Passed = 0
GROUP BY CTE.SqlInstance

Todays failures by instance and context

In [8]:
; With CTE AS (
SELECT 
DetailID,
RIGHT(Context, charindex(' ', reverse(Context) + ' ') - 1) AS SqlInstance
FROM dbachecks.Prod_dbachecks_detail
) 
SELECT 
CTE.SqlInstance,
d.Context,
COUNT(d.DetailID) AS 'Failed Checks'
FROM dbachecks.Prod_dbachecks_detail d
JOIN CTE
ON CTE.DetailID = d.DetailID
JOIN
dbachecks.Prod_dbachecks_summary s
ON
d.SummaryID = S.SummaryID
WHERE
s.TestDate = Convert(date, GetDate())
AND
d.Passed = 0
GROUP BY CTE.SqlInstance,d.Context
ORDER BY CTE.SqlInstance, [Failed Checks] DESC

Number of failures by date, instance and context

In [9]:
; With CTE AS (
SELECT 
DetailID,
RIGHT(Context, charindex(' ', reverse(Context) + ' ') - 1) AS SqlInstance
FROM dbachecks.Prod_dbachecks_detail
) 
SELECT 
s.TestDate,
CTE.SqlInstance,
d.Context,
COUNT(d.DetailID) AS 'Failed Checks'
FROM dbachecks.Prod_dbachecks_detail d
JOIN CTE
ON CTE.DetailID = d.DetailID
JOIN
dbachecks.Prod_dbachecks_summary s
ON
d.SummaryID = S.SummaryID
WHERE
d.Passed = 0
GROUP BY s.TestDate,CTE.SqlInstance,d.Context
ORDER BY CTE.SqlInstance, [Failed Checks] DESC

This is just a quick example of some of the things in the dbachecks ValidationResults database - What more can you do ?